In [1]:
import tensorflow as tf

node1 = tf.constant(10, dtype=tf.float32)
node2 = tf.constant(20, dtype=tf.float32)

node3 = node1 + node2

# 그래프를 실행시키기 위해서는 session이라는게 필요해요(runner)
sess = tf.Session()  # 2.x버전에서는 session이 삭제되었어요!

print(sess.run(node3))

30.0


In [4]:
# Data Flow Graph에 입력값을 주려면 어떻게 해야 하나요?
# placeholder를 이용해요!

import tensorflow as tf

node1 = tf.placeholder(dtype=tf.float32)
node2 = tf.placeholder(dtype=tf.float32)

node3 = node1 + node2

sess = tf.Session()

result = sess.run(node3, feed_dict={node1 : 10,
                                    node2 : 20})
print(result)

30.0


In [1]:
# Tensorflow를 이용해서 Simple Linear Regression을 구현
import numpy as np
import pandas as pd
import tensorflow as tf

# 1. training data set
x_data = (np.array([1,2,3,4,5])).reshape(5,1)   # 공부시간 
t_data = (np.array([3,5,7,9,11])).reshape(5,1)  # 시험성적
  
# 2. placeholder   
X = tf.placeholder(shape=[None,1], dtype=tf.float32)
T = tf.placeholder(shape=[None,1], dtype=tf.float32)

# 3. Weight & bias   
W = tf.Variable(tf.random.normal([1,1]), name='weight')
b = tf.Variable(tf.random.normal([1]), name='bias')

# 4. Hypothesis or predict model
H = tf.matmul(X,W) + b   # y = Wx + b => 2차원행렬로 처리 => y = X dot W + b

# 5. W,b를 구하기 위해 평균제곱오차를 이용한 최소제곱법을 통해
#    loss function을 정의
loss = tf.reduce_mean(tf.square(H - T))

# 6. train
train = tf.train.GradientDescentOptimizer(learning_rate=1e-3).minimize(loss)

# 7. session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())   # 초기화 (2.x 넘어오면서 삭제)

# 8. 학습을 진행
# 반복학습을 진행 ( 1 epoch : training data set 전체를 이용하여 1번 학습하는 것)
for step in range(30000):
    
    _, W_val, b_val, loss_val = sess.run([train,W,b,loss], 
                                         feed_dict={X : x_data, T : t_data})
    
    if step % 3000 == 0:
        print('W : {}, b : {}, loss : {}'.format(W_val, b_val, loss_val))
        
# 9. 학습이 종료된 후 최적의 W와 b가 계산되고 이를 이용한 model이 완성
#    prediction(예측)
result = sess.run(H, feed_dict={X : [[9]]})
print('예측값은 : {}'.format(result))

W : [[-0.6718051]], b : [-0.13792363], loss : 102.87015533447266
W : [[2.0371153]], b : [0.866004], loss : 0.0032703527249395847
W : [[2.0134661]], b : [0.951395], loss : 0.00043031160021200776
W : [[2.0048919]], b : [0.9823527], loss : 5.672912811860442e-05
W : [[2.0017762]], b : [0.99358636], loss : 7.492568784073228e-06
W : [[2.0006516]], b : [0.99764943], loss : 1.006370212053298e-06
W : [[2.0002372]], b : [0.9991496], loss : 1.3190128811402246e-07
W : [[2.0000985]], b : [0.9996586], loss : 2.151323386101467e-08
W : [[2.000052]], b : [0.9998292], loss : 5.604397390612803e-09
W : [[2.000052]], b : [0.9998292], loss : 5.604397390612803e-09
예측값은 : [[19.000296]]


In [16]:
# Simple Linear Regression을 python으로 구현해 보아요!

import numpy as np

# 1. training data set
x_data = np.array([1,2,3,4,5]).reshape(5,1)
t_data = np.array([3,5,7,9,11]).reshape(5,1)

# 2. Weight & bias
W = np.random.rand(1,1)   
b = np.random.rand(1)

# 3. Hypothesis
def predict(x):
    
    y = np.dot(x,W) + b   
    
    return y

# 4. loss_function
def loss_func(input_obj):
    # input_obj : [W, b]
    
    input_W = input_obj[0]
    input_b = input_obj[1]
    
    y = np.dot(x_data,input_W) + input_b
    
    return np.mean(np.power((t_data - y),2))

# 5. 편미분을 위한 함수
def numerical_derivative(f,x):
    
    # f : 미분하려고 하는 다변수 함수
    # x : 모든 값을 포함하는 numpy array  ex) f'(1.0, 2.0) = (8.0, 15.0)
    #     [W, b] 
    delta_x = 1e-4
    derivative_x = np.zeros_like(x)    # [0 0]
    
    it = np.nditer(x, flags=['multi_index'])
    
    while not it.finished:
        
        idx = it.multi_index   # 현재의 iterator의 index를 추출 => tuple형태로 나와요
#         print('현재의 idx : {}'.format(idx))        
        
        tmp = x[idx]     # 현재 index의 값을 잠시 보존.
                         # delta_x를 이용한 값으로 ndarray를 수정한 후 편미분을 계산
                         # 함수값을 계산한 후 원상복구를 해 줘야 다음 독립변수에
                         # 대한 편미분을 정상적으로 수행할 수 있어요!
#         print('현재 temp : {}'.format(tmp))   
        x[idx] = tmp + delta_x        
        fx_plus_delta = f(x)    # f([1.00001, 2.0])   => f(x + delta_x)
        

        x[idx] = tmp - delta_x
        fx_minus_delta = f(x)    # f([0.99999, 2.0])   => f(x - delta_x)
        
        derivative_x[idx] = (fx_plus_delta - fx_minus_delta) / (2 * delta_x)
        
        x[idx] = tmp
        
        it.iternext()
        
    return derivative_x

# learning rate 설정
learning_rate = 1e-4

# 학습을 진행
for step in range(300000):
    input_param = np.concatenate((W.ravel(), b.ravel()), axis=0)  # [W b]
    derivative_result = learning_rate * numerical_derivative(loss_func,input_param)
    
    W = W - derivative_result[:1].reshape(1,1)  # W 갱신
    b = b - derivative_result[1:]               # b 갱신

    if step % 30000 == 0:
        print('W : {}, b : {}'.format(W,b))

        

W : [[0.20905127]], b : [0.62528613]
W : [[1.988679]], b : [1.04087241]
W : [[1.99589439]], b : [1.01482255]
W : [[1.99851108]], b : [1.00537546]
W : [[1.99946004]], b : [1.00194943]
W : [[1.99980418]], b : [1.00070697]
W : [[1.99992899]], b : [1.00025639]
W : [[1.99997425]], b : [1.00009298]
W : [[1.99999066]], b : [1.00003372]
W : [[1.99999661]], b : [1.00001223]


In [ ]:
# sklearn을 이용해 구현해보아요 => 정답확인용

import numpy as np
from sklearn import linear_model

# 1. training data set
x_data = np.array([1,2,3,4,5]).reshape(5,1)
t_data = np.array([3,5,7,9,11]).reshape(5,1)

# 2. linear regression model을 생성
model = linear_model.LinearRegression()

# 3. 학습진행
model.fit(x_data,t_data)

# 4. loss_function
print('W : {}, b : {}'.format(model.coef_,model.intercept_))
    
#5. predict
print(model.predict(([9])))

In [ ]:
# 세가지 방법으로 구현
# python은 정확도 떨어짐
# sklearn은 딥러닝에서 구현 불가
# => 주로 tensorflow